In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [2]:
#train_df=pd.read_pickle('./data/train_df.pickle')
#test_df=pd.read_pickle('./data/test_df.pickle')
train_df=pd.read_pickle('./data/train_df_nopunc.pickle')
test_df=pd.read_pickle('./data/test_df_nopunc.pickle')

In [3]:
comment_lengths=list()
comment_list=train_df['text'].tolist()
for i in tqdm(range(len(comment_list))):
    comment=comment_list[i]
    temp1=len(comment.split())
    comment_lengths.append(temp1)

    

100%|████████████████████████████| 3600000/3600000 [00:27<00:00, 129831.57it/s]


In [4]:
comment_lengths_test=list()
comment_list_test=test_df['text'].tolist()
for i in tqdm(range(len(comment_list_test))):
    comment=comment_list_test[i]
    temp1=len(comment.split())
    comment_lengths_test.append(temp1)

100%|██████████████████████████████| 400000/400000 [00:03<00:00, 131758.07it/s]


In [5]:
#word2vec_simple_from_tf=np.load('./data/word2vec_simple.npz')
word2vec_simple_from_tf=np.load('./data/word2vec_1M_FT.npz')
#word2vec_simple_from_tf=np.load('./data/word2vec_200k_amazon.npz')

In [6]:
embeddings=word2vec_simple_from_tf['arr_0']
word_dict=word2vec_simple_from_tf['arr_1'].tolist()
reverse_dict=word2vec_simple_from_tf['arr_2'].tolist()
embedding_size=embeddings.shape[1]
comment_vector_size=embedding_size*max(comment_lengths)
num_comments=len(comment_list)
train_labels=(train_df['label']).tolist()
n_embeddings,d_embeddings=embeddings.shape

In [7]:
comments_with_word_indices=np.zeros(shape=(num_comments,max(comment_lengths)),dtype='int32')
for i in tqdm(range(len(comment_list))):
    comment=comment_list[i]
    comment_words=comment.split()
    for j,word in enumerate(comment_words):
        comments_with_word_indices[i,j]=word_dict.get(word,0)

100%|█████████████████████████████| 3600000/3600000 [03:18<00:00, 18165.69it/s]


In [8]:
def generate_batch(batch_size):
    batch_indices=random.sample(range(num_comments),batch_size)
    batch_inputs=comments_with_word_indices[batch_indices,:]
    batch_labels=np.reshape(np.asarray([train_labels[idx] for idx in batch_indices])-1,(batch_size,1))
    return batch_inputs, batch_labels, batch_indices

In [9]:
#sanity check
a=generate_batch(2) 
b=a[0][0]
index1=a[2][0]
c=[o for o in b if o!=0]
print(len(c))
print(len(comment_list[index1].split()))
label1=a[1][0]
print(label1)
print(train_df['label'][index1])
print(b)
print(comment_list[a[2][0]])

38
40
[0]
1
[  132  1093   619   104 39289  8780    65     5     2  2501    59   174
    48     0   174    29   163    35   915   135   989   990     4     4
  1147  2401     0    12   144   462  6577  2501     5    30    82   139
    20  1598     2   316     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0   

In [10]:
#creating test dataset
num_comments_test=len(comment_list_test)
test_labels=(test_df['label']-1).tolist()
test_dataset=np.zeros((num_comments_test,max(comment_lengths)),dtype='int32')
for i in tqdm(range(num_comments_test)):
    comment=comment_list_test[i]
    comment_as_word=comment.split()
    for j,word in enumerate(comment_as_word):
        word_index=word_dict.get(word,0)
        test_dataset[i,j]=word_index

100%|███████████████████████████████| 400000/400000 [00:22<00:00, 17954.77it/s]


In [11]:
import time
num_steps=1001
batch_size=64
graph=tf.Graph()
batch_timing=0
run_timing=0
restore_sess=1
test_batch_size=int(num_comments_test/100)
validation_batch_size=500
#model_name="./ckpt/LR_word2vec.ckpt"
model_name="./ckpt/LR_word2vec_1M_FT_nopunc.ckpt"
#model_name="./ckpt/LR_word2vec_200k_Amazon.ckpt"
steps_display=min(int((num_steps-1)/20),1000)
with graph.as_default():
    inputs=tf.placeholder(tf.int32, shape=(batch_size,max(comment_lengths)))
    labels=tf.placeholder(tf.float64, shape=(batch_size,1))
    embedding_matrix=tf.placeholder(tf.float64, shape=(n_embeddings,d_embeddings))
    test_inputs=tf.placeholder(tf.int32, shape=(test_batch_size,max(comment_lengths)))
    validation_inputs=tf.placeholder(tf.int32, shape=(validation_batch_size,max(comment_lengths)))
    W = tf.Variable(tf.random_uniform([comment_vector_size, 1],-1.0,1.0,dtype=tf.float64))
    b = tf.Variable(tf.ones([1],dtype=tf.float64))
    def model(data,batch_size=batch_size):
        batch_embeddings=tf.reshape(tf.nn.embedding_lookup(embedding_matrix,data),[batch_size,max(comment_lengths)*d_embeddings])
        y_matmul=tf.matmul(batch_embeddings,W)+b
        return y_matmul
    y_pred_train=model(inputs,batch_size=batch_size)
    y_pred_test=model(test_inputs,batch_size=test_batch_size)
    y_pred_validation=model(validation_inputs,batch_size=validation_batch_size)
    test_predictions=tf.round(tf.sigmoid(y_pred_test))
    train_predictions=tf.round(tf.sigmoid(y_pred_train)) 
    validation_predictions=tf.round(tf.sigmoid(y_pred_validation)) 
    x_entropy=tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred_train,labels=labels)
    loss =tf.reduce_mean(x_entropy)#+0.01*(tf.reduce_sum(tf.multiply(W,W))+tf.reduce_sum(tf.multiply(b,b)))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    init = tf.global_variables_initializer()
    saver=tf.train.Saver()

with tf.Session(graph=graph) as sess:
    init.run()
    if restore_sess==1:
        try:
            saver.restore(sess,model_name)
        except:
            print('Unexpected Error: model cannot be restored')
    print('initialized')
    loss_val_sum=0
    for i in range(num_steps):
        time1=time.time()
        batch_inputs,batch_labels,_=generate_batch(batch_size)
        time2=time.time()
        feed_dict={inputs:batch_inputs,labels:batch_labels,embedding_matrix:embeddings}
        _,loss_val,train_preds=sess.run([optimizer,loss,train_predictions],feed_dict=feed_dict)
        time3=time.time()
        batch_timing += time2-time1
        run_timing += time3-time2
        loss_val_sum += loss_val
        if i % steps_display==0 and i!=0:
            validation_batch,validation_labels,_=generate_batch(validation_batch_size)
            validation_preds=validation_predictions.eval({validation_inputs:validation_batch,embedding_matrix:embeddings})
            print('Step:',i,
                  ',Average loss in',steps_display, 'steps:',"{0:.3f}".format(loss_val_sum/steps_display),',batch time:'
                  ,"{0:.2f}".format(batch_timing),',Run time:',"{0:.2f}".format(run_timing)
                  ,'Batch Accuracy:', "{:4.2f}".format(100*accuracy_score(batch_labels,train_preds))
                  ,'Valid accuracy:',"{:4.2f}".format(100*accuracy_score(validation_labels,validation_preds)))            
            batch_timing=0
            run_timing=0
            loss_val_sum=0
    y_pred=np.zeros((num_comments_test,1))
    for i in tqdm(range(int(num_comments_test/test_batch_size))):
        test_batch=test_dataset[i*test_batch_size:(i+1)*test_batch_size,:]
        y_pred_batch=test_predictions.eval(feed_dict={test_inputs:test_batch,embedding_matrix:embeddings})
        y_pred[i*test_batch_size:(i+1)*test_batch_size,:]=y_pred_batch            
    Weights=W.eval()
    bias=b.eval()
    try:
        saver.save(sess,model_name)
        print('Model saved!')
    except:
        print('Model could not be saved')

INFO:tensorflow:Restoring parameters from ./ckpt/LR_word2vec_1M_FT_nopunc.ckpt
initialized
Step: 50 ,Average loss in 50 steps: 0.317 ,batch time: 0.02 ,Run time: 1.27 Batch Accuracy: 87.50 Valid accuracy: 88.40
Step: 100 ,Average loss in 50 steps: 0.317 ,batch time: 0.00 ,Run time: 1.14 Batch Accuracy: 92.19 Valid accuracy: 89.80
Step: 150 ,Average loss in 50 steps: 0.295 ,batch time: 0.02 ,Run time: 1.13 Batch Accuracy: 95.31 Valid accuracy: 86.80
Step: 200 ,Average loss in 50 steps: 0.324 ,batch time: 0.02 ,Run time: 1.16 Batch Accuracy: 92.19 Valid accuracy: 85.20
Step: 250 ,Average loss in 50 steps: 0.297 ,batch time: 0.00 ,Run time: 1.18 Batch Accuracy: 90.62 Valid accuracy: 85.80
Step: 300 ,Average loss in 50 steps: 0.305 ,batch time: 0.00 ,Run time: 1.18 Batch Accuracy: 89.06 Valid accuracy: 88.40
Step: 350 ,Average loss in 50 steps: 0.326 ,batch time: 0.02 ,Run time: 1.16 Batch Accuracy: 79.69 Valid accuracy: 85.00
Step: 400 ,Average loss in 50 steps: 0.319 ,batch time: 0.00 ,R

100%|████████████████████████████████████████| 100/100 [01:35<00:00,  1.05it/s]


Model saved!


In [12]:
y_true=test_labels

accuracy=accuracy_score(y_true,y_pred)
precision=average_precision_score(y_true,y_pred)
recall=recall_score(y_true,y_pred)
f1=f1_score(y_true,y_pred)
print('Accuracy: {:4.2f}%, Precision: {:4.2f}%, Recall: {:4.2f}%, F1_score: {:4.2f}%'.format(accuracy*100,precision*100,recall*100,f1*100))

Accuracy: 86.82%, Precision: 82.13%, Recall: 86.20%, F1_score: 86.73%
